In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
%aimport trans.data
%aimport trans.gtrans
%aimport trans.reg
%aimport trans.date_manip


import pandas as pd
idx = pd.IndexSlice

from trans.data import GetData
from trans.gtrans import *
from trans.reg import Reg
import datetime as dt
import dateutil.parser as dup
import datedelta

from trans.date_manip import Date_Manipulator

gd = GetData()

In [3]:
price_df = gd.load_data("price.pkl")

In [4]:
price_df.index

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2018-02-06', '2018-02-07', '2018-02-08', '2018-02-09',
               '2018-02-12', '2018-02-13', '2018-02-14', '2018-02-15',
               '2018-02-16', '2018-02-20'],
              dtype='datetime64[ns]', name='Dt', length=4562, freq=None)

In [5]:
from trans.date_manip import Date_Manipulator
dm = Date_Manipulator( price_df.index )
monthly = dm.periodic( dup.parse("02/16/2018"), datedelta.datedelta(months=1) )

end_of_monthly =  dm.periodic_end_of_month( dup.parse("01/15/2018"))
end_of_monthly[-10:]

End 2018-01-15 00:00:00, last 2018-01-31 00:00:00


[datetime.datetime(2017, 4, 30, 0, 0),
 datetime.datetime(2017, 5, 31, 0, 0),
 datetime.datetime(2017, 6, 30, 0, 0),
 datetime.datetime(2017, 7, 31, 0, 0),
 datetime.datetime(2017, 8, 31, 0, 0),
 datetime.datetime(2017, 9, 30, 0, 0),
 datetime.datetime(2017, 10, 31, 0, 0),
 datetime.datetime(2017, 11, 30, 0, 0),
 datetime.datetime(2017, 12, 31, 0, 0),
 datetime.datetime(2018, 1, 31, 0, 0)]

In [6]:
eom_in_idx = dm.periodic_in_idx_end_of_month( dup.parse("12/15/2017"))
eom_in_idx[-10:]

End 2017-12-15 00:00:00, last 2017-12-31 00:00:00


[Timestamp('2017-02-28 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-04-28 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-09-29 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00')]

In [7]:
month_r = dm.range_in_index( eom_in_idx)
month_r[-10:]

[(Timestamp('2017-02-01 00:00:00'), Timestamp('2017-02-28 00:00:00')),
 (Timestamp('2017-03-01 00:00:00'), Timestamp('2017-03-31 00:00:00')),
 (Timestamp('2017-04-03 00:00:00'), Timestamp('2017-04-28 00:00:00')),
 (Timestamp('2017-05-01 00:00:00'), Timestamp('2017-05-31 00:00:00')),
 (Timestamp('2017-06-01 00:00:00'), Timestamp('2017-06-30 00:00:00')),
 (Timestamp('2017-07-03 00:00:00'), Timestamp('2017-07-31 00:00:00')),
 (Timestamp('2017-08-01 00:00:00'), Timestamp('2017-08-31 00:00:00')),
 (Timestamp('2017-09-01 00:00:00'), Timestamp('2017-09-29 00:00:00')),
 (Timestamp('2017-10-02 00:00:00'), Timestamp('2017-10-31 00:00:00')),
 (Timestamp('2017-11-01 00:00:00'), Timestamp('2017-11-30 00:00:00'))]

In [8]:
price_monthly_df = price_df.loc[ eom_in_idx,: ]
price_monthly_df.shape

(215, 5481)

## Create monthly returns

In [9]:
price_attr = "Adj Close"
price_attr_shift_1d = price_attr + " -1d"
price_attr_shift_1m = price_attr + " -1m"

ret_attr = "Pct"
monthly_ret_attr = "Month " + ret_attr
monthly_rank_attr = monthly_ret_attr + ' rank'

In [10]:
monthly_ret_pl = GenRetAttrTransformer( price_attr, price_attr_shift_1m, monthly_ret_attr, 1 )
monthly_ret_df = monthly_ret_pl.fit_transform( price_monthly_df )
monthly_ret_df.tail()

Adj Close -1m                                                \
                       A        AAL        AAN         AAP        AAPL   
Dt                                                                       
2017-07-31     59.057308  50.108418  38.843315  116.445259  142.942795   
2017-08-31     59.535259  50.227917  46.212559  111.870949  147.617554   
2017-09-29     64.444252  44.642254  44.205486   97.778465  163.412552   
2017-10-31     63.926472  47.386246  43.594700   99.139626  153.567932   
2017-11-30     67.879715  46.717709  36.770226   81.690254  168.434494   

                                                                      ...     \
                 ABBV        ABC        ABMD        ABT        ACC    ...      
Dt                                                                    ...      
2017-07-31  70.857895  93.653625  143.300003  47.878185  45.840462    ...      
2017-08-31  68.931221  92.950203  148.089996  48.707672  46.460716    ...      
2017-09-29  74.245750  79.870285  150.800003  50.450768  46.556999    ...      
2017-10-31  87.615898  82.358452  168.600006  52.847527  43.191666    ...      
2017-11-30  89.607231  76.585899  192.919998  53.971893  40.677456    ...      

           Month Pct                                                    \
                 XOM      XRAY       XRX       XYL         Y       YUM   
Dt                                                                       
2017-07-31 -0.008547 -0.043337  0.067525  0.023453  0.031170  0.027486   
2017-08-31 -0.037108 -0.088022  0.052168  0.097360 -0.082450  0.017753   
2017-09-29  0.074021  0.058833  0.039261  0.009022 -0.015566 -0.041786   
2017-10-31  0.016711  0.021067 -0.089516  0.062270  0.022039  0.011411   
2017-11-30  0.008528  0.097265 -0.021445  0.045064  0.032814  0.125389   

                                                    
                 ZBH      ZBRA      ZION       ZTS  
Dt                                                  
2017-07-31 -0.055140  0.011938  0.032111  0.002244  
2017-08-31 -0.058111  0.013468 -0.034052  0.002879  
2017-09-29  0.026799  0.053254  0.080623  0.016906  
2017-10-31  0.038688  0.068245 -0.015261  0.000941  
2017-11-30 -0.037165 -0.048884  0.070278  0.134457  

[5 rows x 1798 columns]

## Create daily returns (need to construct portfolio)

In [11]:
daily_ret_pl = GenRetAttrTransformer( price_attr, price_attr_shift_1d, ret_attr, 1 )
daily_ret_df = daily_ret_pl.fit_transform( price_df )
daily_ret_df.tail()

Adj Close -1d                                                \
                       A        AAL        AAN         AAP        AAPL   
Dt                                                                       
2018-02-13     68.430000  50.090000  39.439999  109.480003  162.710007   
2018-02-14     68.339996  50.869999  39.509998  106.220001  164.339996   
2018-02-15     69.699997  51.070000  40.330002  109.019997  167.369995   
2018-02-16     72.019997  51.669998  43.790001  108.779999  172.990005   
2018-02-20     71.970001  51.580002  43.400002  107.570000  172.429993   

                                                                       ...     \
                  ABBV        ABC        ABMD        ABT        ACC    ...      
Dt                                                                     ...      
2018-02-13  111.930000  89.101730  233.720001  57.410000  35.889999    ...      
2018-02-14  111.860001  97.389336  233.990005  57.639999  35.959999    ...      
2018-02-15  113.080002  96.024673  255.470001  58.279999  35.650002    ...      
2018-02-16  114.900002  97.220001  261.040009  59.500000  36.369999    ...      
2018-02-20  118.599998  99.400002  267.739990  60.169998  36.770000    ...      

                 Pct                                                    \
                 XOM      XRAY       XRX       XYL         Y       YUM   
Dt                                                                       
2018-02-13 -0.001570 -0.006797 -0.026043  0.011678  0.002628  0.000772   
2018-02-14  0.002097  0.003334  0.021255  0.010336  0.020883  0.001414   
2018-02-15 -0.003270  0.009094  0.023833  0.019370  0.002534  0.013996   
2018-02-16  0.004330  0.012652  0.001639  0.006691  0.010686 -0.009117   
2018-02-20 -0.010321 -0.004792  0.015712 -0.006646 -0.011395 -0.002172   

                                                    
                 ZBH      ZBRA      ZION       ZTS  
Dt                                                  
2018-02-13 -0.005063  0.006034 -0.001921  0.000683  
2018-02-14  0.015268  0.017908  0.053130  0.016658  
2018-02-15  0.017757  0.014646  0.004770  0.054257  
2018-02-16  0.005760 -0.002406  0.005477  0.015669  
2018-02-20 -0.014608 -0.001497 -0.001816  0.007776  

[5 rows x 1798 columns]

## Create end-of-month rank (based on monthly return), and use them as rank for each day of following month

## Rank ONLY the tickers in the universe (e.g., NOT SPY)

In [12]:
univ = ["FB", "AAPL", "AMZN", "NFLX", "GOOG"]

In [13]:
non_univ = ["SPY"]
univ = daily_ret_df.loc[:, idx[ret_attr,:]].columns.get_level_values(1).unique().tolist()

len(univ)
univ = list( set(univ) - set(non_univ) )
univ.sort()
len(univ)

899

898

In [14]:
next_period_rank_pl = GenRankEndOfPeriodAttrTransformer(
    monthly_ret_df,
    monthly_ret_attr,
    univ,
    monthly_rank_attr
)
next_period_rank_df = next_period_rank_pl.fit_transform( daily_ret_df )

next_period_rank_df.loc[ "11/25/2017":"12/01/2017"]

Month Pct rank                                                \
                        A    AAL   AAN   AAP   AAPL   ABBV   ABC   ABMD   
Dt                                                                        
2017-11-27          710.0  270.0  18.0  10.0  817.0  481.0  97.0  862.0   
2017-11-28          710.0  270.0  18.0  10.0  817.0  481.0  97.0  862.0   
2017-11-29          710.0  270.0  18.0  10.0  817.0  481.0  97.0  862.0   
2017-11-30          710.0  270.0  18.0  10.0  817.0  481.0  97.0  862.0   
2017-12-01            NaN    NaN   NaN   NaN    NaN    NaN   NaN    NaN   

                          ...                                             \
              ABT    ACC  ...      XOM   XRAY   XRX    XYL      Y    YUM   
Dt                        ...                                              
2017-11-27  468.0  118.0  ...    438.0  465.0  72.0  715.0  475.0  403.0   
2017-11-28  468.0  118.0  ...    438.0  465.0  72.0  715.0  475.0  403.0   
2017-11-29  468.0  118.0  ...    438.0  465.0  72.0  715.0  475.0  403.0   
2017-11-30  468.0  118.0  ...    438.0  465.0  72.0  715.0  475.0  403.0   
2017-12-01    NaN    NaN  ...      NaN    NaN   NaN    NaN    NaN    NaN   

                                        
              ZBH   ZBRA   ZION    ZTS  
Dt                                      
2017-11-27  592.0  737.0  267.0  347.0  
2017-11-28  592.0  737.0  267.0  347.0  
2017-11-29  592.0  737.0  267.0  347.0  
2017-11-30  592.0  737.0  267.0  347.0  
2017-12-01    NaN    NaN    NaN    NaN  

[5 rows x 898 columns]

## Create HML portfolio return

In [15]:
wt_attr = "HML weight"
portret_col = "Port"
hi_rank, lo_rank = 401, 100

portret_pl = make_pipeline(  GenRankToPortRetTransformer(
    ret_attr,
    next_period_rank_df,
    monthly_rank_attr,
    lambda s: (s >= hi_rank) * 1.0 + (s <= lo_rank) * -1.0,
    wt_attr,
    portret_col
)
                          )

port_ret_df = portret_pl.fit_transform( daily_ret_df )

port_ret_df.loc[ "11/25/2017":"12/01/2017"]

HML weight                                              ...   \
                    A  AAL  AAN  AAP AAPL ABBV  ABC ABMD  ABT  ACC ...    
Dt                                                                 ...    
2017-11-27        1.0  0.0 -1.0 -1.0  1.0  1.0 -1.0  1.0  1.0  0.0 ...    
2017-11-28        1.0  0.0 -1.0 -1.0  1.0  1.0 -1.0  1.0  1.0  0.0 ...    
2017-11-29        1.0  0.0 -1.0 -1.0  1.0  1.0 -1.0  1.0  1.0  0.0 ...    
2017-11-30        1.0  0.0 -1.0 -1.0  1.0  1.0 -1.0  1.0  1.0  0.0 ...    
2017-12-01        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...    

                 Pct                                                    \
                 XOM      XRAY       XRX       XYL         Y       YUM   
Dt                                                                       
2017-11-27 -0.003807  0.007258  0.009126 -0.000739  0.012895  0.005052   
2017-11-28  0.006904  0.008529 -0.027276  0.013010  0.016495  0.018221   
2017-11-29  0.007347 -0.003208 -0.010345 -0.004670  0.020542  0.009626   
2017-11-30  0.012398 -0.019748 -0.012287  0.016716  0.000959  0.020291   
2017-12-01  0.000000 -0.000000  0.000000 -0.000000 -0.000000 -0.000000   

                                          
                 ZBH      ZBRA ZION  ZTS  
Dt                                        
2017-11-27  0.002830 -0.003665  0.0  0.0  
2017-11-28  0.016051  0.005610  0.0 -0.0  
2017-11-29  0.011371  0.000823  0.0  0.0  
2017-11-30  0.004978  0.008133  0.0  0.0  
2017-12-01 -0.000000  0.000000  0.0  0.0  

[5 rows x 1801 columns]

## Find the col.s with the different type of Port returns
### Port: sum of weights * returns.  Weights don't sum up to anything in particular so results can be large
### Port > 0: sum of weights * returns/count, all conditional on weights > 0
### Port < 0: sum of weights * returns/count, all conditional on weights < 0
### Port net: sum of (Port > 0) + (Port < 0).  Sum of the return of a long portfolio and a short portfolio.  Differs from Port in that each side can have a different count as well

In [16]:
cols = port_ret_df.columns.get_level_values(1).unique().tolist()
port_cols = [ c for c in cols if re.search(portret_col,c)]
port_cols

port_ret_df.loc[ "11/25/2017":"12/01/2017", idx[ret_attr, port_cols] ]

['Port', 'Port < 0', 'Port > 0', 'Port net']

Pct                              
                Port  Port < 0  Port > 0  Port net
Dt                                                
2017-11-27 -0.304823  0.002802 -0.001194  0.001608
2017-11-28  4.029888 -0.017795  0.011856 -0.005939
2017-11-29 -1.710224 -0.020424  0.000678 -0.019746
2017-11-30  3.518071 -0.000738  0.007330  0.006592
2017-12-01  0.000000       NaN       NaN       NaN

In [17]:
port_ret_df.shape

(4562, 1801)